In [19]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

# Load Your Trained Model

In [20]:
model = load_model('best_model.h5')

# Define the Color Detection Function

In [21]:
def detect_hsv_color(img):
    # Convert to HSV color space
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    # Focus on the hue channel
    hue_hist = cv2.calcHist([hsv_img], [0], None, [180], [0, 180])
    dominant_hue = np.argmax(hue_hist)

    # Map the dominant hue to a color name
    if dominant_hue < 30 or dominant_hue >= 150:
        return "red"
    elif 30 <= dominant_hue < 90:
        return "yellow"
    elif 90 <= dominant_hue < 150:
        return "green"
    else:
        return "blue"


# Define the Number Prediction Function

In [22]:
def preprocess_and_predict(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (64, 64))
    normalized = resized / 255.0
    reshaped = np.reshape(normalized, (1, 64, 64, 1))
    result = model.predict(reshaped)
    class_index = np.argmax(result)
    return class_index


# Set Up Webcam Capture

In [23]:
cap = cv2.VideoCapture(0)  # Open the default camera

try:
    while True:
        ret, frame = cap.read()  # Read the frame from the webcam
        if not ret:
            break

        # Detect color
        color = detect_hsv_color(frame)

        # Predict number
        number = preprocess_and_predict(frame)

        # Display the results on the frame
        cv2.putText(frame, f'Color: {color}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'Number: {number}', (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Display the resulting frame
        cv2.imshow('UNO Card Detector', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):  # Exit on pressing 'q'
            break
finally:
    cap.release()  # When everything done, release the capture
    cv2.destroyAllWindows()


1/1 [==============================] - 0s 49ms/step


KeyboardInterrupt: 